# 1. Loading & counting

In [1]:
import pandas as pd
import os

In [35]:
#get excel file
xlsx = pd.read_excel('~/wd_speciesid/data/raw/BtF-information/Beyond_the_Fence_ALL_TAGS_to_Feb22.xlsx',
                     header=1)
xlsx

,RecordID,Image Name,Site,Treatment,Date,Time,Moon Phase,Temp,Comments,Number,Pouch status,Species
0,BFGrid01874,2021-05-24 17-58-33 M 1_3.JPG,PCAM01,Park,2021-05-24,17:58:33,Waxing Gibbous,21,NaN,2.0,NaN,Red Kangaroo
1,BFGrid01880,2021-05-24 18-02-50 M 2_3.JPG,PCAM01,Park,2021-05-24,18:02:50,Waxing Gibbous,21,NaN,1.0,NaN,Red Kangaroo
2,BFGrid01881,2021-05-24 22-38-03 M 1_3.JPG,PCAM01,Park,2021-05-24,22:38:03,Waxing Gibbous,16,NaN,2.0,NaN,Red Kangaroo
3,BFGrid01890,2021-05-24 21-27-09 M 1_3.JPG,PCAM01,Park,2021-05-24,21:27:09,Waxing Gibbous,17,NaN,2.0,NaN,Red Kangaroo
4,BFGrid01891,2021-05-24 21-26-03 M 3_3.JPG,PCAM01,Park,2021-05-24,21:26:03,Waxing Gibbous,17,NaN,2.0,NaN,Red Kangaroo
...,...,...,...,...,...,...,...,...,...,...,...,...
15351,BFGrid14116,2021-12-27 01-44-43 M 1_3.JPG,WCAM15,WTZ,2021-12-27,01:44:43,Last Quarter,23,NaN,1.0,NaN,Red Kangaroo
15352,BFGrid14117,2021-12-27 03-05-32 M 1_3.JPG,WCAM15,WTZ,2021-12-27,03:05:32,Last Quarter,22,NaN,1.0,NaN,Red Kangaroo
15353,BFGrid14494,2022-01-14 23-20-32 M 1_3.JPG,WCAM15,WTZ,2022-01-14,23:20:32,Waxing Gibbous,23,NaN,1.0,NaN,Red Kangaroo
15354,BFGrid14534,2022-01-15 02-37-22 M 3_3.JPG,WCAM15,WTZ,2022-01-15,02:37:22,Waxing Gibbous,19,NaN,1.0,NaN,Red Kangaroo


In [3]:
xlsx['Species'].unique()

array(['Red Kangaroo', 'Kangaroo', 'Dingo', 'Rabbit', 'Cat', 'Emu',
       'Bird', 'Pig', 'Euro', 'Fox', 'Echidna', 'Western Grey Kangaroo',
       'Small mammal', 'Other', 'Goat'], dtype=object)

In [4]:
xlsx['Species'].value_counts()

Red Kangaroo             8776
Cat                      2378
Kangaroo                 1375
Rabbit                   1218
Bird                      539
Small mammal              457
Western Grey Kangaroo     165
Dingo                     131
Emu                       120
Euro                       56
Pig                        50
Fox                        35
Goat                       25
Echidna                    18
Other                      13
Name: Species, dtype: int64

# 2 Deciding how to split

In [18]:
#check the dates
for species in xlsx['Species'].unique():
    new_df = xlsx[xlsx['Species']==species]
    min_date = min(new_df['Date'])
    max_date = max(new_df['Date'])
    print(species, ': min date is', min_date, '& max date is', max_date)


Red Kangaroo : min date is 2021-02-10 00:00:00 & max date is 2022-02-17 00:00:00
Kangaroo : min date is 2021-02-11 00:00:00 & max date is 2022-02-12 00:00:00
Dingo : min date is 2021-04-03 00:00:00 & max date is 2022-02-17 00:00:00
Rabbit : min date is 2021-02-07 00:00:00 & max date is 2022-02-15 00:00:00
Cat : min date is 2021-02-11 00:00:00 & max date is 2022-02-18 00:00:00
Emu : min date is 2021-02-14 00:00:00 & max date is 2022-02-15 00:00:00
Bird : min date is 2021-02-15 00:00:00 & max date is 2021-10-04 00:00:00
Pig : min date is 2021-03-31 00:00:00 & max date is 2021-12-14 00:00:00
Euro : min date is 2021-06-09 00:00:00 & max date is 2022-01-24 00:00:00
Fox : min date is 2021-03-20 00:00:00 & max date is 2021-07-26 00:00:00
Echidna : min date is 2021-03-14 00:00:00 & max date is 2021-12-19 00:00:00
Western Grey Kangaroo : min date is 2021-04-23 00:00:00 & max date is 2021-12-16 00:00:00
Small mammal : min date is 2021-02-14 00:00:00 & max date is 2022-02-13 00:00:00
Other : min 

In [158]:
train_entries = xlsx[xlsx['Date']<'2021-10-01 00:00:00'] #Feb2021 to Sep2021
print(len(train_entries))
val_entries = xlsx[ (xlsx['Date']>'2021-09-30 00:00:00') & (xlsx['Date']<'2021-12-01 00:00:00') ] #Oct2021 to Nov2021
print(len(val_entries))
test_entries = xlsx[xlsx['Date']>'2021-11-30 00:00:00'] #from Dec2021
print(len(test_entries))

10029
3089
2238


In [23]:
all_species = xlsx['Species'].unique()

In [11]:
import pandas as pd
import numpy as np

# Initialize dictionaries to store train, validation, and test data
train_data = []
val_data = []
test_data = []

# Loop through each species group
for species in xlsx['Species'].unique():
    # Sort images by date
    new_df = xlsx[xlsx['Species']==species]
    group = new_df.sort_values('Date')

    # Calculate split sizes
    total_size = len(group)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split images
    train_data.append(group[:train_size])
    val_data.append(group[train_size:train_size + val_size])
    test_data.append(group[train_size + val_size:])


In [37]:
train_df = pd.concat(train_data)
val_df = pd.concat(val_data)
test_df = pd.concat(test_data)

In [38]:
total_counts = xlsx['Species'].value_counts()

# Calculate the number of occurrences for each species in the train, val, and test sets
train_counts = train_df['Species'].value_counts()
val_counts = val_df['Species'].value_counts()
test_counts = test_df['Species'].value_counts()

# Calculate the ratios
train_ratios = train_counts / total_counts
val_ratios = val_counts / total_counts
test_ratios = test_counts / total_counts

# Combine the ratios into a single DataFrame
ratios_df = pd.DataFrame({
    'Train Ratio': train_ratios,
    'Validation Ratio': val_ratios,
    'Test Ratio': test_ratios
})

print(ratios_df)


                       Train Ratio  Validation Ratio  Test Ratio
Red Kangaroo              0.699977          0.149954    0.150068
Cat                       0.699748          0.149706    0.150547
Kangaroo                  0.699636          0.149818    0.150545
Rabbit                    0.699507          0.149425    0.151067
Bird                      0.699443          0.148423    0.152134
Small mammal              0.698031          0.148796    0.153173
Western Grey Kangaroo     0.696970          0.145455    0.157576
Dingo                     0.694656          0.145038    0.160305
Emu                       0.700000          0.150000    0.150000
Euro                      0.696429          0.142857    0.160714
Pig                       0.700000          0.140000    0.160000
Fox                       0.685714          0.142857    0.171429
Goat                      0.680000          0.120000    0.200000
Echidna                   0.666667          0.111111    0.222222
Other                    

# ?. Moving the images around

In [22]:
#create parent directory for processed images
image_path = '/media/jess/DATA2/PhD/data/wild_deserts/'
new_folder = 'processed/'
processed_path = os.path.join(image_path, new_folder)
try:    
    os.mkdir(processed_path)
    print('Done!')
except:
    print('Folder created already!')

Done!


In [31]:
#create a folder for each species
def create_species_folder(species):
    species_folder = species + '/'
    species_path = os.path.join(processed_path, species_folder)
    try:    
        os.mkdir(species_path)
        print(species + '- Done!')
    except:
        print('Folder created already!')

In [32]:
species = xlsx['Species'].unique()
for sp in species:
    create_species_folder(sp)

Red Kangaroo- Done!
Kangaroo- Done!
Dingo- Done!
Rabbit- Done!
Cat- Done!
Emu- Done!
Bird- Done!
Pig- Done!
Euro- Done!
Fox- Done!
Echidna- Done!
Western Grey Kangaroo- Done!
Small mammal- Done!
Other- Done!
Goat- Done!
